In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
changeupDL = pd.read_csv('DraftLeagueChangeup.csv')
changeupMLB = pd.read_csv('MLBchangeup.csv')

In [3]:
changeupMLB = changeupMLB[changeupMLB['pitches'] > 50]
changeupMLB.drop('pitches', inplace=True, axis=1)

In [4]:
changeupDL = changeupDL.astype(str)
changeupMLB = changeupMLB.astype(str)

In [5]:
changeupDL['league'] = 'Draft League'
changeupMLB['league'] = 'MLB'

In [6]:
pitchersDL = list(changeupDL['pitcher'])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
def get_index_from_pitcher(pitcher):
    return changeup[changeup.pitcher == pitcher].index[0]

In [9]:
def get_pitcher_from_index(index):
    return changeup[changeup.index == index].values[0][0]

In [10]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        changeup = pd.concat([changeupDL, changeupMLB])

        changeup["combined_features"] = changeup.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(changeup["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        changeup = changeup[(changeup['pitcher'] == pitcher_like) | (changeup['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)
        
        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass
    

In [11]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp = cosine_comp.sort_values('MLB DL Pitcher')
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
51,"Alba, Max","Knehr, Reiss",0.301511
128,"Alba, Miguel",No Comp,No Similarity
88,"Allain, Luke","Smith, Riley",0.3
5,"Ammerlaan, Ashe","Borucki, Ryan",0.286039
34,"Ayers, Kyle","Minor, Mike",0.210819
...,...,...,...
92,"Wilhite, Thomas","Lynch, Daniel",0.33541
38,"Wilson, Benjamin","Weber, Ryan",0.365148
71,"Wilson, Jeffrey","López, Pablo",0.33541
50,"Wright, Kerry","Foltynewicz, Mike",0.316228


In [12]:
cosine_comp.to_csv('changeup_cosine_comp.csv', sep='\t')